In [2]:
from backend.arena.solr_parser import parse_solr_response
from backend.arena import solr_query
import pysolr

solr_url = "http://localhost:8983/solr/lasso_quickstart"
solr = pysolr.Solr(solr_url, always_commit=True)

In [3]:
from backend.lql.antlr_parser import parse_interface_spec

lql_string = """
    Calculator {
        Calculator(int)->None
        mean(list)->float
    }
    """

interfaceSpecification = parse_interface_spec(lql_string)

Interface Specification: InterfaceSpecification(className=Calculator, constructors=[MethodSignature(methodName=Calculator, returnType=None, parameterTypes=['int'])], methods=[MethodSignature(methodName=mean, returnType=float, parameterTypes=['list'])])


In [4]:
import importlib
importlib.reload(solr_query)

solr_url = "http://localhost:8983/solr/lasso_quickstart"
solr = pysolr.Solr(solr_url)
solr_query = solr_query.translate_to_solr_query(interfaceSpecification)
print("QUERY:", solr_query)

QUERY: (method:mean AND (methodSignatureParamsOrderedNodefault:*pt_list*)) OR (method:mean AND count_positional_non_default_args:(1))&group=true&group.field=module


In [ ]:
results = solr.search(solr_query)
print(f"Found {len(results)} results")

allModulesUnderTest = parse_solr_response(results)
moduleUnderTest = allModulesUnderTest[0]